# Essentials to better Python

**Overview**
 * Packages and error-handling examples
 * List comprehensions
 * Decorators

# Error handling
How to raise an Exception. Let's look at an example from the array class, here defined as a function for representation. 
```python
def array(shape, *values):
    """Error example from the array class"""
    arr = []    
    n = len(values)        
    for val in values:
        arr.append(val)
    return arr

A = array((1,6),2,3,4,5,6)
print(A)
```



# Self-made packages

You can make your own python packages with (several) subpackages:

1) Create a new directory for your package (e.g. MyPackage)

2) Create a new folder "MyPackage" within the folder MyPackage

4) Create a module myModule.py located in MyPackage/MyPackage/myModule.py

4) Create a setup.py file, located in (the first) MyPackage

5) Create (empty)  \_\_init\_\_.py files in all folders. 

6) run "pip install ." from the (first) directory MyPackage

7) See if you can import MyPackage in python when you are not located inside the MyPackage directory
```python
from distutils.core import setup
import setuptools
setup(
    name='my_package',
    version='1.0',
    author='Vegard',
    author_email='vegard@simula.no',
    packages=setuptools.find_packages(),
)
```



# Interactive programming: Make your own package

Follow the recipe below. Let myModule.py be a script containing one function my_name() that returns your name (10 minutes). 

1) Create a new directory for your package (e.g. MyPackage)

2) Create a new folder "MyPackage" within the folder MyPackage

4) Create a module myModule.py located in MyPackage/MyPackage/myModule.py

4) Create a setup.py file, located in (the first) MyPackage

5) Create (empty)  \_\_init\_\_.py files in all folders. 

6) run "pip install ." from the (first) directory MyPackage

7) See if you can import and use MyPackage in python/ipython. 

```python
from distutils.core import setup
import setuptools
setup(
    name='my_package',
    version='1.0',
    author='Vegard',
    author_email='vegard@simula.no',
    packages=setuptools.find_packages(),
)
```

```python
import numpy
import setuptools
from setuptools.extension import Extension
from Cython.Build import cythonize
with open("README.md", "r") as fh:
    long_description = fh.read()   
extensions=[
    Extension("cython_color2gray", ["instapy/cython_color2gray.pyx"],
    include_dirs=[numpy.get_include()],
    ), Extension("cython_color2sepia", ["instapy/cython_color2sepia.pyx"],
    include_dirs=[numpy.get_include()])
]
setuptools.setup(
    name="instapy",
    version="0.0.13",
    author="Vegard Vinje",
    author_email="vegarvi@ifi.uio.no",
    description="Instagram filters in Python",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="https://github.uio.no/IN3110/IN3110-vegarvi/",
    packages=setuptools.find_packages(),
    scripts=["bin/instapy"],
    ext_modules=cythonize(extensions),
    setup_requires=["cython", "numpy", "setuptools>=18.0"],
    install_requires=["numpy", "numba", "opencv-python"],
    classifiers=[
        "Programming Language :: Python :: 3",
        "License :: OSI Approved :: MIT License",
        "Operating System :: OS Independent",
    ],
    python_requires='>=3.6',
)
```

# List comprehensions

List comprehensions provide a compact and readible way to create lists. 


**Syntax**:

Create a list without list comprehension:

```python
from math import sin
old_list = [0.1, 0.3, -0.4, 0.2]
def filter(x):
    if x > 0:
        return True
    else:
        return False
    
new_list = []
for x in old_list:
    if filter(x):
        new_list.append(sin(x))
```        
the same task with list comprehension

```python
new_list = [sin(x) for x in old_list if filter(x)]
```

### Example 1: List of even numbers

**Task**: Create a list of even numbers.

**Solution** without list comprehension:

In [34]:
def is_even(i):
    return i%2==0

even_numbers = []
for i in range(20):
    if is_even(i):
        even_numbers.append(i)
print(even_numbers)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


**Solution** with list comprehension:

In [35]:
even_numbers = [i for i in range(20) if i%2==0]
even_numbers

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

### Example 2: Remove sensitive information from log data

**Task**: Remove all strings in a logfile that contain passwords

**Solution** without list comprehension:

In [36]:
fp = open("log.txt", "r")

log = []
for line in fp:
    if "password" not in line:
        log.append(line.strip())
fp.close() 

log

['09.Sept 2021 14:30: New user enters webpage',
 '09.Sept 2021 14:31: Login email: vegard@simula.no',
 '09.Sept 2021 14:35: User leaves webpage']

**Solution** with list comprehension:

In [37]:
with open('log.txt', "r") as fp:
    log = [line.strip() for line in fp if "password" not in line]
    
log    

['09.Sept 2021 14:30: New user enters webpage',
 '09.Sept 2021 14:31: Login email: vegard@simula.no',
 '09.Sept 2021 14:35: User leaves webpage']

## Functions as arguments

Like all objects, functions can be arguments to functions

In [8]:
def add(x,y):
    return x+y

def sub(x, y):
    return x-y

def apply(func, x, y):
    return func(x, y)

In [9]:
apply(add, 1, 2)

3

In [10]:
apply(sub, 7, 5)

2

## Functions inside functions

Python allows nested function definitions:

In [11]:
def g(x, y):
    
    def cube(x):
        return x*x*x
    
    return y*cube(x)

g(4, 6) 

384

## Function returning functions

In [12]:
def h():
    pi = 0.13
    def inner_h():
        print("Inside inner_h but can access pi={}".format(pi))
        
    return inner_h

foo = h()
foo

<function __main__.h.<locals>.inner_h>

In [13]:
foo()

Inside inner_h but can access pi=0.13


## More functions returning functions: *decorators*

A toy example

In [16]:
def foo():
    return 1

def outer(func):
    def inner():
        print("before calling function")
        return func() + 100
    return inner

decorated = outer(foo)

The function `decorated` is a decorated version of function `foo`.
It is `foo` plus something more:

In [17]:
decorated()

before calling function


101

To simplify, we could just write
```python 
foo = outer(foo)
```
to replace foo with its decorated version each time it is called

## A (slightly) more useful decorator

Suppose we have been given a function that only works for some numerical inputs:

In [17]:
from math import log
def f(x):
    return log(x) - 2  # Not defined for x<=0

In [18]:
f(5)

-0.3905620875658997

In [19]:
f(-1)

ValueError: math domain error

Suppose we want to limit the range of values sent to this function:

The idea is that we **wrap** the function inside another function:

## Interactive programming (15 minutes)

1) Implement the normal function f(x) in a python script

2) Create a decorator-function checkrange that calls f, but prints a custom message to the user if x <= 0. Hint: The decorator function chekcrange should return a function, and not a function call. 

3) Optional: Perform a test using a test function (with assert) checking that your function works as intended

In [5]:
from math import log

def checkrange(func):
    """Provides a safe version of f. Avoids math domain error."""
    #def inner
        #...
        # return ...
    #return inner

def f(x):
    return log(x) - 2  # Not defined for x<=0

In [6]:
def checkrange(func):
    def inner(x):
        if x <= 0:
            print("Error: x must be larger than zero")
        else:
            return func(x)
    return inner

In [7]:
f_safe = checkrange(f)
f_safe(5)

-0.3905620875658997

In [8]:
f_safe(-1)

Error: x must be larger than zero


Voilà!!

## The `@decorator` syntax

Python provides a short notation for decorating a function with
another function:

In [26]:
@checkrange
def g(x):
    return log(x) - 2

In [27]:
g(0)

Error: x must be larger than zero


This is essentially the same as writing `g = checkrange(g)`.

A decorator is simply a function taking a function as input
and returning another function. 

The syntax `@decorator` is a
short-cut for the more explicit `f = decorator(f)`.

## A (much) more useful decorator: memoization

The first time we learned multiplication, our strategy might to add cumulatively: e.g. 3x3 = 3 + 3 + 3 = 6 + 3 = 9

In [1]:
from time import sleep

def slow_mult(x,y):
    res = 0
    for i in range(y):
        print("Thinking...")
        sleep(1)
        res += x
    return res

print(slow_mult(3,3))
print(slow_mult(3,3))

Thinking...
Thinking...
Thinking...
9
Thinking...
Thinking...
Thinking...
9


We call the function with the same input arguments, and hence perform the same (slow) calculations multiple times.

The idea of memoization (or buffering) is to buffer the input-output pairs for which the function was called.
If the function is called twice with same input arguments, we return the buffer value.

The implementation of a memoization with a `decorator` could look like:

In [2]:

def memoize(func):
    ''' Caches a function's return value each time it is called.
        If called later with the same arguments, the cached value is returned
        (not reevaluated). '''
    cache = {}  # Stores all input-output pairs

    def inner(x, y):
        if (x, y) in cache:
            return cache[(x, y)]
        else:
            result = func(x, y)
            cache[(x, y)] = result
            return result
        
    return inner

Now we can apply the decorator to our slow function. Demo:

In [54]:
@memoize
def slow_mult(x, y):
    print("Thinking...")
    sleep(1)     # Simulate a long computation
    return x*y

@memoize
def slow_add(x, y):
    print("Thinking...")
    sleep(1)     # Simulate a long computation
    return x+y

... and test it out

In [59]:
slow_mult(3, 6)

18

## Decorator summary 

* A function that takes a function as argument and returns a modified function
* `@decorator` syntax simply a short cut for the standard function call `f = decorator(f)`.

## PEP8: How to write more Pythonic code

Clear and consistent style is critical for writing "good code".

* Python comes with an extensive programming style guidline: **PEP8**.
* It consists of a list of do's and dont's for writing Python.
* Get familiar with the conventions once, and you will automatically start using them.
* I will give you some examples below

### Guide to Pythonic code: Bindary operations

* Add whitespaces around bindary mathematical operations:

```python
# Do:
x = x + 1

# Don't:
x=x+1
```


### Guide to Pythonic code: Naming conventions 


* For **variables**:

```python
# Do
shopping_list = ["Bananas", "Apples"]
gravity_acceleration = 9.81
# Don't
ListOfStudents = ["Bananas", "Apples"]
GRAVITYACCELERATION = 9.91  
```

* For **functions**:
    
```python
def order_items(image):
    pass
```

* For **classes**:

```python
# Do:
class ElectricCar:
    pass

# Don't:
class electriccar:
    pass
```


### Guide to Pythonic code: Indentations and spacing


* Aways use **four** white spaces when indenting (set your editor accordingly):

```python
# Do
def order_items(image):
    pass  # Four whitespaces


# Don't 
def order_items(image):
  pass    # Not four whitespaces
```

* Break long lines "nicely":

```python
# Do:
shopping_list = {"Apple": 2, "Banana": 10, "Chocolate": 1,
                 "Toothpaste": 1, "Shampoo": 2}

# Don't: second line is under-indented
shopping_list = {"Apple": 2, "Banana": 10, "Chocolate": 1, "Toothpaste": 1, "Shampoo": 2}
```

### Guide to Pythonic code: flake8

You can use the flake8 command to verify that your code follows the PEP convention.

**Demo** on shopping.py